In [ ]:
from pathlib import Path

In [ ]:
__file__ = "openwrt.ipynb"
TopDir = Path(__file__).parent.absolute()
print(TopDir)
ProjectName = 'tests'


In [ ]:
# internal var
ProjectDir = TopDir.joinpath(ProjectName)
PackageInfoFile:Path = ProjectDir.joinpath('tmp', '.packageinfo')
TargetInfoFile:Path = ProjectDir.joinpath('tmp', '.targetinfo')
LogFile:Path = ProjectDir.joinpath('logs/package/feeds/packages/ffmpeg/full/compile.txt')
print(PackageInfoFile)
print(TargetInfoFile)
print(LogFile)

## packages info lexer

In [ ]:
import re
import json
from src.lexer.packageinfo import PackageInfoLexer
from src.parser.packageinfo import PackageInfoParser

testFullString = PackageInfoFile.read_text()
### hack some code to pass through laxer
testFullString = re.sub('\t\t\t/sys/module/iwlwifi/parameters/debug','\t\t  /sys/module/iwlwifi/parameters/debug',testFullString)
testFullString = re.sub('\t\t\t  % echo 0x43fff > /sys/module/iwlwifi/parameters/debug','\t\t    % echo 0x43fff > /sys/module/iwlwifi/parameters/debug',testFullString)
testFullString = re.sub('\t\t\t  drivers/net/wireless/intel/iwlwifi/iwl-debug.h','\t\t    drivers/net/wireless/intel/iwlwifi/iwl-debug.h',testFullString)

m = PackageInfoLexer()
m.build(debug=False)           # Build the lexer
p = PackageInfoParser()
p.build(m.lexer)
packageInfoAst = p.gen_ast(testFullString)
with open(ProjectDir.joinpath('package.ast.json').as_posix(),'w') as f:
    f.write(json.dumps(packageInfoAst,indent=2,sort_keys=True))
# print(p.test(testFullString))
# m.testSlient(testFullString)     # Test it
print('Done')


In [ ]:
from src.lexer.targetinfo import TargetInfoLexer
from src.parser.targetinfo import TargetInfoParser
import ply.yacc as yacc
# from src.ast.targetinfo import *
testFullString = TargetInfoFile.read_text()


m = TargetInfoLexer()
m.build(debug=False)           # Build the lexer
p = TargetInfoParser()
p.build(lexer=m.lexer)
targetInfoAst = p.gen_ast(testFullString)
with open(ProjectDir.joinpath('target.ast.json').as_posix(),'w') as f:
    f.write(json.dumps(targetInfoAst,indent=2,sort_keys=True))
# print(p.test(testFullString))
# m.lexer.input(testFullString)
# m.testSlient(testFullString)     # Test it
print('Done')


In [ ]:
from src.lexer.logs import LogLexer
from src.parser.logparser import LogParser
import json
testFullString = LogFile.read_text()



m = LogLexer()
m.build(debug=False)           # Build the lexer
p = LogParser()
p.build(lexer=m.lexer)
LogAst = p.gen_ast(testFullString)
with open(ProjectDir.joinpath('log.ast.json').as_posix(),'w') as f:
    f.write(json.dumps(LogAst,indent=2,sort_keys=True))
# print(p.test(testFullString))
# m.lexer.input(testFullString)
# m.testSlient(testFullString)     # Test it
print('Done')
